# Field Analysis

> Latest stats are from 2021, either because scimago stopped collecting data, or newer data is deemed unreliable regarding citations

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("journal.csv")

In [3]:
def get_cats(df):
    _cs = df["categories"].str.split("; ")
    idk = "sourceid"
    _df = pd.DataFrame(
        {"catbase": _cs.sum(), idk: np.repeat(df[idk].values, _cs.str.len())}
    )
    return pd.concat([_df, _df["catbase"].str.extract("(.*) \((.*)\)")], axis=1).assign(
        field=lambda df: np.where(df.loc[:, 0].isna(), df["catbase"], df.loc[:, 0])
    ).rename(columns={1: "Q"}).loc[:, ["field", "Q", idk]]

In [4]:
cat_base = get_cats(df.head(10000)).merge(df)

In [5]:
field_pivot = (
    cat_base.fillna("no Q")
    .pivot_table(index="field", columns="Q", values="sourceid", aggfunc="count")
    .fillna(0)
    .assign(s=lambda df: df.sum(axis=1))
    .sort_values("s", ascending=False)
)
field_pivot.head(15).style.set_caption("Count values by fields")

Q,Q1,Q2,Q3,Q4,miscellaneous,no Q,s
field,,,,,,,
Medicine (miscellaneous),612.000000,611.000000,5.000000,0.000000,0.000000,0.000000,1228.000000
Education,337.000000,166.000000,0.000000,0.000000,0.000000,0.000000,503.000000
Computer Science Applications,174.000000,174.000000,4.000000,0.000000,0.000000,3.000000,355.000000
Sociology and Political Science,327.000000,22.000000,0.000000,0.000000,0.000000,0.000000,349.000000
Economics and Econometrics,174.000000,150.000000,0.000000,0.000000,0.000000,0.000000,324.000000
"Ecology, Evolution, Behavior and Systematics",172.000000,127.000000,0.000000,0.000000,0.000000,0.000000,299.000000
Molecular Biology,99.000000,100.000000,99.000000,0.000000,0.000000,0.000000,298.000000
Psychiatry and Mental Health,137.000000,137.000000,17.000000,0.000000,0.000000,0.000000,291.000000
Electrical and Electronic Engineering,168.000000,106.000000,0.000000,0.000000,0.000000,13.000000,287.000000


In [6]:
def draw_table(df):
    return (
        df.pivot_table(index="field", columns="Q")
        .loc[field_pivot.head(15).index]
        .loc[:, lambda df: df.isna().mean() < 0.5]
        .style.background_gradient(axis=0)
    )

In [7]:
(
    cat_base.groupby(["field", "Q"])[["h_index", "journal_rating", "total_docs_3years"]]
    .mean()
    .pipe(draw_table)
    .set_caption("Mean values of fields")
)

In [8]:
def gini(s):
    vc = s.value_counts(normalize=True)
    diffs = np.abs(vc.values.reshape(-1, 1) - vc.values.reshape(1, -1))
    return diffs.sum() / (2 * vc.shape[0] ** 2 * vc.mean())

def top5(s):
    return s.value_counts(normalize=True).head(5).sum()

## Concentration metrics by fields

In [9]:
cat_base.groupby(["field", "Q"])[["country", "publisher"]].agg([gini, top5]).pipe(draw_table).set_caption("Concentration metrics by fields")